In [159]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT
from vit_pytorch import SimpleViT

In [160]:
print(f"Torch: {torch.__version__}")

Torch: 2.2.1


In [161]:
print("loading image dataset")
images = torch.load("boards2.pt")
print("loading label dataset")
labels = torch.load("labels2.pt")
print(len(images), len(labels))

loading image dataset
loading label dataset
100014 100014


In [162]:
# Hyperparameters:
batch_size = 64
epochs = 30
lr = 5e-4
gamma = 0.7
seed = 142
IMG_SIZE = 8
patch_size = 2
num_classes = 10
dim = 128

In [163]:
device = 'cuda'

In [164]:
class CustomChessDataset(Dataset):
    def __init__(self):
        pass

    def __len__(self):
        return len(labels)

    def __getitem__(self, idx):
        label = labels[idx]
        img = images[idx]
        return img, label

In [165]:
import torch.utils.data as data

full_dataset = CustomChessDataset()

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

print("Train size: ", train_size)
print("Test size: ", test_size)

train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Data Loaders:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Train size:  80011
Test size:  20003


In [166]:
# Linear Transformer:
efficient_transformer = Linformer(
    dim=dim,
    seq_len= 65,
    depth=12,
    heads=8,
    k=64
)

# Vision Transformer Model:
model = ViT(dim=dim, image_size=IMG_SIZE, patch_size=patch_size, num_classes=num_classes, transformer=efficient_transformer, channels=1).to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# Learning Rate Scheduler for Optimizer:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [167]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

  0%|          | 0/1251 [00:00<?, ?it/s]

Epoch : 1 - loss : 1.6715 - acc: 0.3140 - val_loss : 1.6015 - val_acc: 0.3391


  0%|          | 0/1251 [00:00<?, ?it/s]

Epoch : 2 - loss : 1.5430 - acc: 0.3665 - val_loss : 1.5435 - val_acc: 0.3671


  0%|          | 0/1251 [00:00<?, ?it/s]

...

Epoch : 28 - loss : 0.2300 - acc: 0.8985 - val_loss : 2.0217 - val_acc: 0.5329


  0%|          | 0/1251 [00:00<?, ?it/s]

Epoch : 29 - loss : 0.2296 - acc: 0.8990 - val_loss : 1.9382 - val_acc: 0.5384


  0%|          | 0/1251 [00:00<?, ?it/s]

Epoch : 30 - loss : 0.2194 - acc: 0.9024 - val_loss : 1.9730 - val_acc: 0.5393


In [168]:
torch.save(model, "model2.pt")
print("model saved")

model saved


In [169]:
import chess.pgn
from create_dataset import chess_board_to_image
import io



model = torch.load("model2.pt")

pgn = open("Nakamura.pgn")

game = chess.pgn.read_game(pgn)
header = chess.pgn.read_headers(pgn)
print(header)
#while not game.is_end():
game = game.end()
#game = game.next()
tensor = chess_board_to_image(str(game.board()))
tensor.unsqueeze_(0)
with torch.no_grad():
    predictions = model(tensor.to(device))
    print(predictions)
    probabilities = F.softmax(predictions, dim=1) 
    np_arr = predictions.detach().cpu().numpy()
    
        

Headers(Event='Wch U10', Site='Cannes', Date='1997.??.??', Round='2', White='Nakamura, Hikaru', Black='El Mikati, Mohamad', Result='1-0', WhiteElo='', BlackElo='', ECO='C11')
tensor([[-5.5200, -2.2672, -5.5602, -2.9908,  6.6583, -4.0547, -0.7927,  0.6678,
          6.3818,  5.6270]], device='cuda:0')
